<a href="https://colab.research.google.com/github/Dagobert42/langID-NLP/blob/main/langID_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>